# Mount Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Append project path

In [9]:
import sys

PROJECT_PATH_UTIL = "/content/drive/MyDrive/Colab Notebooks/SCVQA/util"
#                    /content/drive/MyDrive/.../util
sys.path.append(PROJECT_PATH_UTIL)

# Import libraries and py script coded

In [10]:
import torch
from torch import nn

import os
import random
import numpy as np
import pandas as pd
from pathlib import Path

from model import LSTM, Transformer

# Custom function

In [11]:
def up_scale(score_predicted: float, mos_max: float, mos_min: float):
  return score_predicted * (mos_max - mos_min) + mos_min

# Setup parameters

In [12]:
_LSTM = "LSTM"
_TRANSFORMER = "Transformer"

_CSCVQ = "CSCVQ"
_SCVD = "SCVD"

_ResNet18 = "ResNet18"
_ResNet34 = "ResNet34"
_ResNet50 = "ResNet50"
_ResNet101 = "ResNet101"
_ResNet34_ResNet50 = "ResNet34_ResNet50"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#===============================================================================

# Instance
MODEL = _TRANSFORMER          # Model Architecture Approach
DATABASE = _CSCVQ             # Database
CNN_EXTRACTION = _ResNet50    # Extractor

# Input video
VIDEO_NAME = "sc_ChromeBrowsing_1280x720_30_8bit_420_300f_264QP48.yuv"

#===============================================================================



FEATURE_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/feature/{DATABASE}/{CNN_EXTRACTION}/")
#                   /content/drive/MyDrive/.../feature/{DATABASE}/{CNN_EXTRACTION}/

MODEL_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/model/{MODEL}/{DATABASE}/{CNN_EXTRACTION}/")
#                  /content/drive/MyDrive/.../model/{MODEL}/{DATABASE}/{CNN_EXTRACTION}/

MODEL_DIR_HIST_FILE = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/model/{MODEL}/{DATABASE}/{CNN_EXTRACTION}/history.csv")
#                            /content/drive/MyDrive/.../model/{MODEL}/{DATABASE}/{CNN_EXTRACTION}/history.csv


MOS_MIN, MOS_MAX = (20.1179, 74.0773) if DATABASE == _SCVD else (20.53108285, 72.75596394)


print(
    f"[{MODEL}-based] | database: {DATABASE}, CNN extraction: {CNN_EXTRACTION}, device: {DEVICE}"
)

[Transformer-based] | database: CSCVQ, CNN extraction: ResNet50, device: cuda


# Read input video data

In [13]:
input_video_file = f"{FEATURE_DIR}/{VIDEO_NAME}/feature.npy"

input_video = np.load(input_video_file)
input_video = torch.from_numpy(input_video)

print(
    f"Read {VIDEO_NAME} from {DATABASE}"
)

Read sc_ChromeBrowsing_1280x720_30_8bit_420_300f_264QP48.yuv from CSCVQ


# Inference

In [14]:
feature_size = None
if CNN_EXTRACTION == _ResNet18 or CNN_EXTRACTION == _ResNet34:
    feature_size = 1024
elif CNN_EXTRACTION == _ResNet50 or CNN_EXTRACTION == _ResNet101:
    feature_size = 4096
else:
    feature_size = 5120

model = None
if MODEL == _LSTM:
    model = LSTM(
        device=DEVICE,
        feature_size=feature_size,
    ).to(device=DEVICE)
elif MODEL == _TRANSFORMER:
    model = Transformer(
        device=DEVICE,
        feature_size=feature_size,
    ).to(device=DEVICE)

if os.path.exists(MODEL_DIR_HIST_FILE):
    hist_df = pd.read_csv(MODEL_DIR_HIST_FILE)
    model_file = Path(MODEL_DIR / hist_df["DIR"].iloc[-1] / "model.pt")
    if os.path.exists(model_file):
        print(f"Load model from {model_file}")
        model.load_state_dict(torch.load(f=str(model_file)))

score_predicted = None

model.eval()
with torch.inference_mode():

  input = input_video.to(device=DEVICE)
  input = input.unsqueeze(dim=0)

  score_predicted = model(input).item()
  score_predicted = up_scale(score_predicted, MOS_MIN, MOS_MAX)

#Summary
print()
print("Result")
print(f"Model: {MODEL}-based")
print(f"Database: {DATABASE}")
print(f"Extractor: {CNN_EXTRACTION}")
print()
print(f"Video: {VIDEO_NAME}")
print(f"Predict score: {round(score_predicted, 2)}")

Load model from /content/drive/MyDrive/Colab Notebooks/SCVQA/model/Transformer/CSCVQ/ResNet50/20240321_172018/model.pt

Result
Model: Transformer-based
Database: CSCVQ
Extractor: ResNet50

Video: sc_ChromeBrowsing_1280x720_30_8bit_420_300f_264QP48.yuv
Predict score: 60.35
